<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       WhitesGeneral function in Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'> The WhitesGeneral() function checks for the presence of correlation among the residual terms after running a regression. The function determines if there exists any heteroscedastic variance in the residuals of regression tests. The output specifies the following:</p>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><code>'ACCEPT'</code> means the null hypothesis is accepted, and there is homoscedasticity variance evident.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><code>'REJECT'</code> means the null hypothesis is rejected, and there is evidence of heteroscedasticity.</li></p>
 
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The Whites-General test does not require reordering like the Goldfeld-Quandt test, and is not sensitive to the normal distribution assumption like the Breusch-Pagan-Godfrey test.</p>
 
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following procedure is an example of how to use WhitesGeneral() function:</p>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Use the function MultivarRegr() for regression testing.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Use WhitesGeneral() on the residual output from MultivarRegr().</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Determine if the variance is homoscedastic or heteroscedastic from the WhitesGeneral() result.</li>
</p>


<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>1. Initiate a connection to Vantage</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In the section, we import the required libraries and set environment variables and environment paths (if required).

In [ ]:
from teradataml import (
    create_context,
    execute_sql,
    load_example_data,
    DataFrame,
    in_schema,
    TDSeries,
    MultivarRegr,
    TDAnalyticResult,
    WhitesGeneral,
    Figure,
    plot,
    db_drop_view,
    remove_context,
    copy_to_sql,
    db_drop_table
    )

from teradatasqlalchemy.types import *

from teradataml import to_numeric
# Modify the following to match the specific client environment settings
display.max_rows = 5

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../../UseCases/startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=PP_WhitesGeneral_Python.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;background-color:#00233C;'>

<p style = 'font-size:18px;font-family:Arial;color:#00233c'><b>1.2 Getting Data for This Demo</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we will get the time series data which is available in the teradataml library and use the same to show the usage of the function.</p>

In [ ]:
load_example_data("uaf", ["house_values"])

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Data Exploration</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Create a "Virtual DataFrame" that points to the data set in Vantage. Check the shape of the dataframe as check the datatype of all the columns of the dataframe.</p>

In [ ]:
df = DataFrame.from_table("house_values")
df

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>3. WhitesGeneral</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The WhitesGeneral() function checks for the presence of correlation among the residual terms after running a regression. The function determines if there exists any heteroscedastic variance in the residuals of regression tests.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The first step is to convert the time series data into TDSeries, which is required for the input time series which are passed to the MultivarRegr function.</p>


In [ ]:
data_series_df = TDSeries(data=df,
                              id="cityid",
                              row_index="TD_TIMECODE",
                              payload_field=["house_val","salary","mortgage"],
                              payload_content="MULTIVAR_REAL")

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Execute multivariate regression function to identify the degree of linearity the explanatory variable and multiple response variables. Generate the model statistics and residual data as well.</p>

In [ ]:
multivar_out = MultivarRegr(data=data_series_df,
                                variables_count=3,
                                weights=False,
                                formula="Y = B0 + B1*X1 + B2*X2",
                                algorithm='QR',
                                coeff_stats=True,
                                model_stats=True,
                                residuals=True)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We execute the Whites-General test on TDAnalyticResult object over the result attribute of the "multivar_out" as input. We will create teradataml TDAnalyticResult object over the 'result' attribute of the 'multivar_out' and use it as input to the WhitesGeneral function.</p>

In [ ]:
data_art_df = TDAnalyticResult(data=multivar_out.result)
 
uaf_out = WhitesGeneral(data=data_art_df,
                            variables_count=3,
                            significance_level=0.05)
 
# Print the result DataFrame.
wh_df = uaf_out.result
wh_df

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>If we scroll to the right we can observe that the NULL hypothesis as <code>'ACCEPT'</code> means the null hypothesis is accepted, and there is homoscedasticity variance evident.</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We can also execute the Whites-General test on TDSeries object over the 'fitresiduals' attribute of the 'multivar_out'. We will create teradataml TDSeries object over the 'fitresiduals' attribute of the 'multivar_out' and use it as input to the WhitesGeneral function</p>

In [ ]:
data_series_df = TDSeries(data=multivar_out.fitresiduals,
                          id="cityid",
                          row_index="ROW_I",
                          row_index_style="SEQUENCE",
                          payload_field=["ACTUAL_VALUE","CALC_VALUE","RESIDUAL"],
                          payload_content="MULTIVAR_REAL")

uaf_out1 = WhitesGeneral(data=data_series_df, 
                         variables_count=3, 
                         significance_level=0.05)

# Print the result DataFrame.
wh_df= uaf_out1.result
wh_df

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>If we scroll to the right we can observe that the NULL hypothesis as <code>'ACCEPT'</code> means the null hypothesis is accepted, and there is homoscedasticity variance evident.</p>


<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>4. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up intermediate tables.</p>

In [ ]:
db_drop_table("house_values")

In [ ]:
remove_context()

<hr style="height:1px;border:none;background-color:#00233C;">

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>WhitesGeneral function reference: <a href = 'https://docs.teradata.com/search/all?query=WhitesGeneral&content-lang=en-US'>here</a></li>
    
</ul>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>